# Silver Layer: ERP Product Categories Transformation
Refining product category metadata from `erp_px_cat_g1v2`.
- **Framework**: `silver_engine` implementation.
- **Transformations**: 
    - Normalizes `maintenance` status from string (Yes/No) to Boolean (True/False).
    - Aligns schema with the Product dimension requirements.
- **Output**: Delta table `workspace.silver.erp_product_category`.

In [0]:
%run ../../helpers/silver_engine.ipynb

In [0]:
%python
import pyspark.sql.functions as F

def logic(df):
    return (
        df
        # 1. Boolean Normalization: Mapping "YES" to True and everything else to False
        .withColumn("maintenance_flag", 
            F.when(F.upper(F.col("maintenance")) == "YES", F.lit(True))
             .otherwise(F.lit(False))
        )
        # 2. Schema Alignment: Renaming columns to business-friendly names
        .withColumnRenamed("id", "category_id")
        .withColumnRenamed("cat", "category")
        .withColumnRenamed("subcat", "subcategory")
        
        # 3. Final Selection: Selecting columns for the downstream Product Dimension
        .select("category_id", "category", "subcategory", "maintenance_flag")
    )

# Executing the automated silver pipeline
run_silver_pipeline("erp_px_cat_g1v2", "erp_product_category", logic)

In [0]:
%sql
-- Final verification of standardized product categories and maintenance flags
SELECT * FROM workspace.silver.erp_product_category;